In [2]:
import numpy as np
import compare_anns
import itertools

#read the fasta file
def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences

#translate from letters to indices
def translate_observations_to_indices(obs):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    return [mapping[symbol] for symbol in obs]

#translate from annotations to indices
def translate_annotations_to_indices(obs):
    mapping = {'N': 0, 'C': 7, 'R': 8}
    return [mapping[symbol] for symbol in obs]

#load into data
data = {}
for i in np.arange(1,11):
    temp={}
    temp[f'genome{str(i)}'] = translate_observations_to_indices(read_fasta_file(f'genome{str(i)}.fa')[f'genome{str(i)}'])
    data.update(temp)
for i in np.arange(1,6):
    temp={}
    temp[f'true-ann{str(i)}'] = translate_annotations_to_indices(read_fasta_file(f'true-ann{str(i)}.fa')[f'true-ann{str(i)}'])
    data.update(temp)
#954265
#809+809+639+639

In [9]:
#group codons
def group_codons(ann):
    i=1
    out=[0]
    while i < len(ann):
        if out[-1]==0 and ann[i]==0:
            out.append(0)
        elif (out[-1]==1 and ann[i]==7) or (out[-1]==2 and ann[i]==7) :
            out.append(2)
            i+=2
        elif (out[-1]==4 and ann[i]==8)  or (out[-1]==5 and ann[i]==8) :
            out.append(5)
            i+=2
        elif (out[-1]==0 and ann[i]==7) :
            out.append(1)
            i+=2
        elif (out[-1]==0 and ann[i]==8) :
            out.append(4)
            i+=2
        elif (out[-1]==2 and ann[i]==0) :
            out[-1]=3
            out.append(0)
        elif (out[-1]==5 and ann[i]==0) :
            out[-1]=6
            out.append(0)
        elif (out[-1]==2 and ann[i]==8) :
            out[-1]=3
            out.append(4)
            i+=2
        elif (out[-1]==5 and ann[i]==7) :
            out[-1]=6
            out.append(1)
            i+=2
        i+=1
    return out

data2=data

for i in np.arange(1,6):
    data2[f"true-ann{str(i)}"]=group_codons(data[f"true-ann{str(i)}"])

In [11]:
triplet_indices=[0,1,2,3]
triplet_indices+=[list(x) for x in itertools.combinations_with_replacement([0,1,2,3],3)]
d=[1,3,3,3,3,3,3]


def count_transitions_and_emissions_var(K, D, x, z):
    """
    Returns a KxK matrix and a KxD matrix containing counts cf. above
    """
    kxk=np.zeros((K,K))
    kxd=np.zeros((K,D))
    count=0
    total=0
    while total < len(x):
        kxd[z[count]][triplet_indices.index(x[total:total+d[z[count]]])]+=1
        total+=d[z[count]]
        count+=1
    for j in np.arange(len(z)-1):
        kxk[z[j]][z[j+1]]+=1
    return kxk,kxd


def training_by_counting_var(K, D, x, z):
    """
    Returns a HMM trained on x and z cf. training-by-counting.
    """
    init_probs=np.zeros(K)
    init_probs[z[0]]=1
    trans_probs,emission_probs=count_transitions_and_emissions_var(K,D,x,z)
    trans_probs/=np.sum(trans_probs,1).reshape(-1,1)
    emission_probs/=np.sum(emission_probs,1).reshape(-1,1)
    return hmm(init_probs,trans_probs,emission_probs)

result=[]
for i in np.arange(1,6):
    four_test=np.delete(np.arange(5),[i-1])
    four_genome=[]
    four_ann=[]
    for j in four_test:
        four_genome.extend(data2[f"genome{str(j+1)}"])
        four_ann.extend(data2[f"true-ann{str(j+1)}"])
    result.append(training_by_counting_var(7,66,four_genome,four_ann))


ValueError: [0] is not in list